This processed was done in Google COlb, so the path will be different

# Import necessary libraries

In [2]:
%pip install pycountry
%pip install pandas
import pandas as pd
import pycountry
import re

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


ImportError: Unable to import required dependencies:
numpy: Error importing numpy: you should not try to import numpy from
        its source directory; please exit the numpy source tree, and relaunch
        your python interpreter from there.

In [22]:
df = pd.read_csv("./rawPhase2.csv")
print(df.shape)

(24490, 7)


# Process entries where the "Country" column is not the name of a country

In [23]:
print(len(list(df["Country"].unique())))
print("-------------------------------")
for i in list(df["Country"].unique()):
  print(i)

109
-------------------------------
USA
Hong Kong
Thailand
China
HK
United States
nan
US
Thailand China
Malaysia
U.S.A.
Indonesia
Italy
France
US/Mexico
Mexico
CoO scratched out
French
Hong Kong, US Patent
USA?
Thailand & Malaysia
USA/Mexico
USA / Mexico
MEXICO
Hong Kong/Mexico
Hong Kong & Mexico
Mexico & Hong Kong
China or Thailand
Malaysia (Boxed)
India
Thail&
7286
Came on a Clock Stand? (Check Gallery)
China or Malaysia
Malasyia
Hong Kong or Malaysia
Maylasia
Malaysa
Inda
Hong Kong & Malaysia
Thailand or China
No Country on base
?
Malaysia or Hong Kong
not on base
Flat Malaysia
Raised Malaysia
China / Thailand
unknown
No Country
Thailand / China
Canada
Thailand / Malaysia
Hong Kong/Malaysia
Malaysia or China
China Malaysia
Malaysia or Thailand
Hong Kong or Malasya
Thailand, Malaysia
Malaysia, Thailand
Malaysia, China
Thailand/Malaysia
Malaysia/Thailand
Macau
Malaysia (common) & China
Malaysia, China or Thailand
Dale Jarrett
China Thailand
Malaysia, China, Thailand
China, Malaysia
Ch

# Guessed based on where the majority of this model is produced

In [24]:
df.loc[df["Country"].isin(["B6502", "B6501", "B6494", "B6495"]), "Country"] = "Thailand"

# Fill in `NaN`, `unknown` and `?` values with strings "N/A"

In [25]:
df["Country"] = df["Country"].fillna("N/A")
df.loc[df["Country"].isin(["?", "unknown"]), "Country"] = "N/A"

# Handle 2026 data (`Toy number` and `Country` columns are mixed up)

In [26]:
df.loc[df["Country"].isin(["JHW18", "JHW17", "JHW46", "JKD99", "JHW19"]), "Country"] = "Thailand"

# Handle `'58 Corvette Coupe`, 1995 version
This is a bit of a special case, the country entry is `"Some are marked Malaysia, some are not marked"`, we will assume all of them are produced in Malaysia in this case

In [27]:
df.loc[df["Country"] == "Some are marked Malaysia, some are not marked", "Country"]= "Malaysia"

# Standardize the Country entry in each row by converting its value to a canonical (consistent and recognized) form.

Ensure that all country names and their variations are uniformly represented, making data analysis and grouping more accurate and straightforward.

For example, entries like `"USA"`, `"U.S.A."`, and `"United States"` are all standardized to `"USA"`, while `"Thailand or Indonesia"` becomes the list `["Thailand", "Indonesia"]`.

In [28]:
# Canonical country list
canonical_map = {
    "USA": {"usa", "u.s.a.", "united states", "us", "usa?", "hong kong, us patent", "us/mexico", "usa/mexico", "usa / mexico"},

    "Mexico": {"mexico", "mexico & hong kong", "hong kong/mexico", "hong kong & mexico"},

    "Hong Kong": {"hong kong", "hk", "hong kong or malaysia", "hong kong & malaysia", "hong kong/malaysia", "hong kong or malasya"},

    "Thailand": {"thailand", "thail&", "thailand & malaysia", "thailand / malaysia", "thailand, malaysia", "thailand/malaysia",

                 "thailand or china", "thailand / china", "thailand china", "china thailand", "thailand, china",
                 "thailand or malaysia", "thailand / indonesia", "thailand/indonesia", "thailand or indonesia",
                 "thailand malaysia", "thailand indonesia", "thailand (painted or engraved on base)", "thailand/china"},

    "China": {"china", "china or thailand", "china / thailand", "china malaysia", "china, malaysia", "china/thailand",
              "china, thailand", "china/ malaysia"},

    "Malaysia": {"malaysia", "malaysia (boxed)", "flat malaysia", "raised malaysia", "malasyia", "maylasia", "malaysa",
                 "malaysia or hong kong", "malaysia or china", "malaysia, china", "malaysia, china or thailand",
                 "malaysia or thailand", "malaysia/thailand", "malaysia / thailand", "malaysia & china",
                 "malaysia, india", "malasia", "malasysia", "mayalsia", "[malaysia", "malaysia (circled)",
                 "malaysia (common) & china", "malaysia/china", "china/malaysia", "malaysia, thailand", "malaysia/thailand",
                 "malaysia, thailand or china", "malaysia or indonesia", "indonesia, malaysia"},

    "India": {"india", "inda"},

    "Indonesia": {"indonesia", "indonesia(?)"},

    "Italy": {"italy"},

    "France": {"france", "french"},

    "Canada": {"canada"},

    "Macau": {"macau"},

    "Vietnam": {"vietnam"},
}

# Preprocess mapping: word set to canonical country name
value_to_country = {}
for canonical, variants in canonical_map.items():
    for v in variants:
        value_to_country[v.strip().lower()] = canonical

# Helper function to extract canonical countries from entry
def normalize_entry(entry):
    entry = str(entry).strip().lower()

    # Remove special characters (except / and , which separate countries)
    cleaned = re.sub(r"[?()\[\]]", "", entry)

    # Split based on common delimiters
    parts = re.split(r"[\/,&]| or | and |,|;", cleaned)
    parts = [p.strip() for p in parts if p.strip()]

    # Use a set to keep track of the normalized answer (so we don't double count permutations)
    result = set()
    for part in parts:
        if part in value_to_country:
            result.add(value_to_country[part])

    # If any valid country matched, return as list
    if result:
        return sorted(result) if len(result) > 1 else list(result)[0]

    # Fallback: mark as unknown
    return "Unknown"

# Apply normalization
df["Country (Normalized)"] = df["Country"].apply(normalize_entry)

# Expand/ Explode rows with multiple countries into separate rows
df = df.explode("Country (Normalized)")

# Convert the year entries into integers
df["Year"] = pd.to_numeric(df["Year"], errors='coerce').astype('Int64')

# Drop entries where the year row is NaN
df= df[df["Year"].notna()]

# Fill in the empty entries of `Series`,`Color`,`Window Color`, and `Interior Color` as "N/A"

In [29]:
df["Series"] = df["Series"].fillna("N/A")
df["Color"] = df["Color"].fillna("N/A")
df["Window Color"] = df["Window Color"].fillna("N/A")
df["Interior Color"] = df["Interior Color"].fillna("N/A")

# Create `Alpha-3 Code` column that contains the alpha-3 code for country names (normalized)

In [30]:
def get_iso_alpha3(country):
  try:
        # Try to look up the country name
        return pycountry.countries.lookup(country).alpha_3
  except:
        return "UNK"  # or "UNK" if you want a placeholder for unknown

# If 'Country (Normalized)' contains strings:
df["Country Code"] = df["Country (Normalized)"].apply(
    lambda x: get_iso_alpha3(x)
)

In [35]:
df.sample(10)

,Year,Model Name,Series,Color,Window Color,Interior Color,Country,Country (Normalized),Country Code
16408,2004,'34 Ford Delivery,Larry Wood's 35th Anniversary Set,Orange,Clear,Black,China,China,CHN
22090,2013,'70 Toyota Celica,HW City: Street Power,Metalflake Dark Green,Black Tint,Black,Malaysia,Malaysia,MYS
20348,2016,2009 Ford F-150,Hot Trucks 5-Pack,Magenta,Tinted (Yellow),Chrome,Malaysia,Malaysia,MYS
22864,2019,'17 Ford F-150 Raptor,Car Culture: Desert Rally 4/5,Metalflake Black,Smoke tint,Black,Thailand,Thailand,THA
21387,2014,File:2011DodgeDemonCarded.jpg,HW Workshop: Heat Fleet,Metalflake Orange,Clear,Black,Malaysia,Malaysia,MYS
11391,1994,MAN Truck,Auto-City,Red,Clear,Black,China,China,CHN
874,1969,Custom Charger,Hot Wheels,Spectraflame Creamy 'Lavender' Pink,Clear,N/A,USA,USA,USA
18213,2007,'70 Mustang Mach 1 (1:50),G-Machines: Street Legal (Series 2) 4/5,Dark Blue,N/A,N/A,China,China,CHN
17771,2012,Cool-One,HW Code Cars 20/22,Metallic Olive,Tinted (Yellow),Green,Malaysia,Malaysia,MYS
18487,2022,Bone Shaker,HW Dream Garage 4/5,ZAMAC,N/A,Chrome,Malaysia,Malaysia,MYS


In [32]:
# Save the result as "processedPhase2.csv"
df.to_csv("./processedPhase2.csv", index=False)